In [1]:
# This is a cell to hide code snippets from displaying
# This must be at first cell!

from IPython.display import HTML

hide_me = ''
HTML('''<script>
code_show=true; 
function code_toggle() {
  if (code_show) {
    $('div.input').each(function(id) {
      el = $(this).find('.cm-variable:first');
      if (id == 0 || el.text() == 'hide_me') {
        $(this).show();
      }
    });
    $('div.output_prompt').css('opacity', 1);
  } else {
    $('div.input').each(function(id) {
      $(this).show();
    });
    $('div.output_prompt').css('opacity', 1);
  }
  code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input style="opacity:0" type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [2]:
def get_all_shots_for_point(df, point_index: int):
    return (df[df["Point"].astype(int)==int(point_index)]).sort_values(by='Shot')

#set the winner of the point for every shot

def set_point_winner(df, point_index: int):
    """
    Returns
    1 if player1 won the point
    0 if player 2
    -1 if player 1 hit his serve out
    -2 if player 2 hit his serve out
    """
    df_all_shots_for_point = get_all_shots_for_point(df, point_index)
    df_last_shot = df_all_shots_for_point[df_all_shots_for_point["Shot"]==df_all_shots_for_point["Shot"].max()]
    last_shot_result = df_last_shot.iloc[0]["Result"]
    index_winner = -1
    str_player_last_shot = df_last_shot.iloc[0]["Player"]
    print(str_player_last_shot)
    if (df_last_shot.iloc[0]["Shot"]==1) and (last_shot_result in ["Out", "Net"]):
        if (str_player_last_shot == "Player"):
            if df_last_shot.iloc[0]["Type"]=="second_serve":
                index_winner = "0" #DF
            else:
                index_winner = "-1" #Player serve out or net
        else:
            if df_last_shot.iloc[0]["Type"]=="second_serve":
                index_winner = "1"#DF
            else:
                index_winner = "-2" #Opponent serve out or net
    elif (str_player_last_shot == "Player"):
        if (last_shot_result in ["In"]):
            index_winner = "1"
        elif (last_shot_result in ["Out", "Net"]):
            index_winner = "0"
        else:
            index_winner = "-1"
    elif (str_player_last_shot == "Opponent"):
        if (last_shot_result in ["In"]):
            index_winner = "0"
        elif (last_shot_result in ["Out", "Net"]):
            index_winner = "1"
        else:
            index_winner = "-1"
    else:
        index_winner = "-1"
    #print(str(index_winner))
    df.loc[df["Point"]==point_index,"Winner_Point"]=index_winner
    
#set_point_winner(df, 8)

In [3]:
def change_ball_in_to_out(x: float, y: float, shot_index: int, server_side: str, server_far_near: str
                          , x_limits, y_limits)->(float, float):
    x_center = x_limits[1]#0
    x_left = x_limits[0]#-4.11
    x_right = x_limits[2]#4.11
    y_center = y_limits[1]#11.88
    y_bottom = y_limits[0]#5.48
    y_top = y_limits[2]#18.48
    x_min, x_max, y_min, y_max=-1000, -1000,-1000,-1000
    if server_far_near=="far":
        y_min=y_bottom
        y_max=y_center
        if server_side=="ad":
            x_max=x_center
            x_min=x_left
        elif server_side=="deuce":
            x_max=x_left
            x_min=x_center
        else:
            raise ValueError ( "Error inchange_ball_in_to_out" )
    elif server_far_near=="near":
        y_max=y_top
        y_min=y_center
        if server_side=="ad":
            x_max=x_left
            x_min=x_center
        elif server_side=="deuce":
            x_max=x_center
            x_min=x_left
        else:
            raise ValueError ( "Error inchange_ball_in_to_out" )
    else:
        raise ValueError ( "Error inchange_ball_in_to_out" )
    if x > x_max:
        x = x_max-0.5
    if x < x_min:
        x = x_min + 0.5
    if y > y_max:
        y = y_max-0.5
    if y < y_min:
        y = y_min + 0.5
    return x,y

"""def change_serve_in_to_out(x: float, y: float, shot_index: int, server_side: str, server_far_near: str)->(float, float):
    x_center = 0
    x_left = -4.11
    x_right = 4.11
    y_center = 11.88
    y_bottom = 5.48
    y_top = 18.48
    x_limits=[x_left, x_center, x_right]
    y_limits=[y_bottom, y_center, y_top]
    return change_ball_in_to_out(x, y, shot_index, server_side, server_far_near, x_limits, y_limits)
"""
def change_shot_in_to_out(x: float, y: float, shot_index: int, server_side: str, server_far_near: str)->(float, float):
    if (shot_index==1):
        x_center = 0
        x_left = -4.11
        x_right = 4.11
        y_center = 11.88
        y_bottom = 5.48
        y_top = 18.48
    else:
        x_center = 0
        x_left = -4.11
        x_right = 4.11
        y_center = 11.88
        y_bottom = 5.48
        y_top = 18.48
    x_limits=[x_left, x_center, x_right]
    y_limits=[y_bottom, y_center, y_top]
    return change_ball_in_to_out(x, y, shot_index, server_side, server_far_near, x_limits, y_limits)

In [4]:
def swap_server_all_shots(df, current_point):
	df_shot1=df[(df["Shot"]==1)&(df["Point"]==current_point)]
	if len(df_shot1) > 0:
		server=df_shot1.iloc[0]["Player"]
		if server=="Player":
			server="Opponent"
			returner="Player"
		else:
			server="Player"
			returner="Opponent"
		df.loc[(df["Point"]==current_point)&(df["Shot"]%2==0),"Player"]=returner
		df.loc[(df["Point"]==current_point)&(df["Shot"]%2==1),"Player"]=server
		df.loc[(df["Point"]==current_point),"comment"]="swapped srv"

In [5]:
#créer une figure
#fig = go.Figure()
def transform_far_to_near(df):
    df["Bounce (x)"]=df.apply(lambda x: -(x["Bounce (x)"]), axis=1)
    df["Bounce (y)"]=df.apply(lambda x: 23.73-(x["Bounce (y)"]), axis=1)
    return df
                              
def trace_rect(fig1, left_double_court_x,right_double_court_x, top_double_court_y ):
    """
    outside lines
    """
    court_x=[left_double_court_x, left_double_court_x, right_double_court_x, right_double_court_x]
    court_y=[0,top_double_court_y, top_double_court_y, 0]
    trace0=go.Scatter(
        x=court_x, y=court_y,
        mode="lines",line=dict(color='white')
    )
    #fig.add_trace(trace0)
    
    #trace0=go.Scatter(
    fig1.add_trace(go.Scatter(
        x=court_x, y=court_y,
        mode="lines",line=dict(color='white')
    ))
    #fig.add_trace(trace0)

def trace_court():
    fig1 = px.line()
    trace_rect(fig1, 0-4.11-1.37, 0+4.11+1.37, 23.73)
    trace_rect(fig1, 0-4.11, 0+4.11, 23.73)
    trace_rect(fig1, 0-4.11, 0+4.11, 11.88)
    trace_rect(fig1, 0-4.11, 0+4.11, 5.48)
    trace_rect(fig1, 0-4.11, 0+4.11, 12.08)
    trace_rect(fig1, 0-4.11, 0+4.11, 18.48)
    return fig1

In [6]:
def apply_corrections(df):
    for index, row in df.iterrows():
        result2 = row["Result2"]
        print(result2)
        if (result2!="")and(str(result2).lower()!="nan"):
            if(result2=="Shot=0"):
                df.loc[index,"Shot"]=0
            else:
                df.loc[index,"ResultOld"]=row["Result"]
                df.loc[index,"Result"]=row["Result2"]
                df.loc[index,"Bounce (x)old"]=row["Bounce (x)"]
                df.loc[index,"Bounce (y)old"]=row["Bounce (y)"]
                df.loc[index,"Bounce (x)"]=row["Bounce (x)2"]
                df.loc[index,"Bounce (y)"]=row["Bounce (y)2"]
            print(index, row)
        
#apply_corrections(df)
#df.to_csv("Gerasimov FAA.csv")

In [7]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')

In [8]:
filename_csv="Isner Pospisil.csv"
df=pd.read_csv(filename_csv, index_col=0)
lst_players=filename_csv.split(".")[0].split(" ")
name1=lst_players[0]
name2=lst_players[1]

In [9]:
# LAUNCH VIDEO PLAYER

#pip install python-vlc
import time, vlc

#http://olivieraubert.net/vlc/python-ctypes/doc/vlc.MediaPlayer-class.html
filename=filename_csv.replace("csv","mp4")#"Gerasimov FAA.mp4"

# method to play video
def video2(source):
    # creating vlc media player object
    media_player = vlc.MediaPlayer()
  
    # media object
    media = vlc.Media(source)

    # setting media to the media player
    media_player.set_media(media)

    # setting video scale
    media_player.video_set_scale(0.6)
    media_player.audio_set_mute(True)
    # setting play rate
    # doubles the speed of the video
    media_player.set_rate(1.5)
    # start playing video
    media_player.play()

    # wait so the video can be played for 5 seconds
    # irrespective for length of video
    time.sleep(5)
    return media_player

# call the video method
player = video2(filename)
#GO to point 1
current_point=0

In [10]:
#WRITE SOME TEXT OVER THE VIDEO
from vlc import VideoMarqueeOption

def write_text_video(player, text):
    # Some marquee examples.  Marquee requires '--sub-source marq' in the
    # Instance() call above, see <http://www.videolan.org/doc/play-howto/en/ch04.html>
    #https://www.olivieraubert.net/vlc/python-ctypes/doc/vlc.VideoMarqueeOption-class.html
    player.video_set_marquee_int(VideoMarqueeOption.Enable, 1)
    player.video_set_marquee_int(VideoMarqueeOption.Size, 24)  # pixels
    #player.video_set_marquee_int(VideoMarqueeOption.Position, Position.Bottom)
    if False:  # only one marquee can be specified
        player.video_set_marquee_int(VideoMarqueeOption.Timeout, 5000)  # millisec, 0==forever
        t = media.get_mrl()  # movie
    else:  # update marquee text periodically
        player.video_set_marquee_int(VideoMarqueeOption.Timeout, 0)  # millisec, 0==forever
        player.video_set_marquee_int(VideoMarqueeOption.Refresh, 1000)  # millisec (or sec?)
    df_point = df[df["Point"]==current_point]
    player.video_set_marquee_string(VideoMarqueeOption.Text, text)

#write_text_video(player,"a")

In [11]:
#VIDEO Go TO next POINT
def go_to_nextpoint(videoplayer, current_point, indexshot):
    """
    Args:
        player(str): "Player" or "Opponent" 
        current_point(int): index of current_point
        indexshot(int): index of shot (1=Serve, -1=Last shot)
    """
    print("go_to_nextpoint " + str(current_point))
    df_point = df[df["Point"]==current_point]
    delay_before_shot = 0
    if len(df_point) > 0:
        if (indexshot == -1):
            shot_index = max(df_point["Shot"])
            delay_before_shot = 1500
        else:
            shot_index = indexshot
        #move video to the time of the point
        #get time of point at indexshot
        time_point=df_point[(df_point["Shot"]==shot_index)].iloc[0]["Video Time"]
        #print("at" + str(time_point) + " sec")
        videoplayer.set_time(round(time_point*1000)+1500-delay_before_shot) #go to 7mn
        #print(df_point.head())
        time = round(player.get_time()/1000)
        server=df_point.iloc[0]["Player"]
        #shot_results = "-".join(list(df_point["Result"]))
        shot_results = ["{0}-{1}; ".format(i+1, df_point.iloc[i]["Result"]) for i in range(len(df_point))]
        text="{3}\n Serve:{0} \n {2} \n {1}".format(server, " ".join(shot_results), df_point.iloc[0]['Winner_Point'], time)
        write_text_video(player, text)
#go_to_nextpoint(player, 1, -1)

In [12]:
#TRACE court with all shots of the point

import ipywidgets as widgets

from IPython.display import display, Javascript
# These lines before the animated plotly figure
#import plotly.io as pio
#pio.renderers.default = 'notebook'
def trace_point(index_point):
    print("trace_point " + str(current_point))
    fig1 = trace_court()
    df_point = df[(df["Point"]==index_point)&(df["Shot"]>0)]
    last_shot = None
    if len(df_point) >0:
        #trace a line from last hit position to last bounce to display the last shot
        last_shot = df_point.iloc[-1]
        #print(last_shot["Hit (x)"])
        df2 = pd.DataFrame(dict(
        x = [last_shot["Hit (x)"],last_shot["Bounce (x)"]],
        y = [last_shot["Hit (y)"],last_shot["Bounce (y)"]]))
        fig4=px.line(df2,x="x",y="y")
        fig4.update_traces(patch={"line": {"width": 1, "dash": 'dot'}})

    #FIG HIT
    fig2=px.scatter(df_point,x="Hit (x)",y="Hit (y)",text="Shot"#, color="Player"
                    #, symbol="Stroke"
                    #, animation_frame="Shot"#, animation_group="Player"
                    #, textposition='top right',textfont=dict(color='#E58606')

                    #, symbol="Spin"
                       #, trendline='ols',trendline_color_override='red'
                        #, marginal_y="rug"
                        #, marginal_x="histogram"
                        #, facet_col="Winner_Point"#, facet_row="time"
                        , color_discrete_map={"Player":"green","Opponent":"red"}
                        , hover_data=["Shot","Stroke"]
                       )
    
    fig2.for_each_trace(lambda t: t.update(mode = 'lines+markers+text', textposition='middle right'
                                           ,textfont=dict(size=10,color='gray')))
    for fr in fig2.frames:
        for d in fr.data:
            d.update(mode='markers+lines+text', textposition='middle right'
                     ,textfont=dict(size=10,color='gray'))
    fig2.update_traces(patch={"line": {"width": 1, "dash": 'dot'}}) 
    """marker_symbol=[1,101], """
    """fig2.add_annotation(
      x=5,  # arrows' head
      y=5,  # arrows' head
      ax=10,  # arrows' tail
      ay=10,  # arrows' tail
      xref='x',
      yref='y',
      axref='x',
      ayref='y',
      text='',  # if you want only the arrow
      showarrow=True,
      arrowhead=3,
      arrowsize=1,
      arrowwidth=1,
      arrowcolor='black'
    )"""
    #FIG Player POSITION
    fig3=px.scatter(df_point,x="Bounce (x)",y="Bounce (y)",color="Result"
                    
                       #, trendline='ols',trendline_color_override='red'
                        #, marginal_y="rug"
                        #, marginal_x="histogram"
                        #, facet_col="Winner_Point"#, facet_row="time"
                        , color_discrete_map={"Net":"black","In":"orange","Out":"black"}
                        , hover_data=["Shot"],text="Shot"
                       )
    fig3.for_each_trace(lambda t: t.update(mode = 'markers+text', textposition='middle right'
                                           ,textfont=dict(size=9, color='gray')))
    
    fig=go.Figure(data=fig1.data + fig2.data + fig3.data)

    #fig.update_layout(title="Serves P1 (point size represents Speed)")
    fig.update_xaxes(
        scaleanchor = "y",
        scaleratio = 2,
      )
    fig.update_layout(xaxis_range=[-5,5], yaxis_range=[-4,11.88*2+4])
    fig.update_layout(
         xaxis=dict(showgrid=False), 
         yaxis=dict(showgrid=False),
        autosize=False,
    width=700,
    height=400
    )
    
    fig.show()
    
#trace_point(current_point)


In [13]:
def get_point_infos(index_point, is_replace_with_names, name1, name2):
    """
    Returns:
        server(str or ""): "Player" or "Opponent" (if is_replace_with_names => name1, name2)
        winner(str or ""): "0" (P2) or "1" (P1) (if is_replace_with_names => name1, name2)
        lst_shots_out(Dataframe or None): dataframe with list of shots where Result=Out or Net
        last_shot(Dataframe or None): dataframe with the row of the max Shot
    """
    df_point = df[df["Point"]==index_point]
    server = ""
    winner = ""
    server_side = ""
    last_shot = None
    df_shots_out_or_net = None
    if len(df_point) > 0:
        df_point=df_point.sort_values(by="Shot")
        df_shot1=df_point[(df_point["Shot"]==1)]
        if len(df_shot1) > 0:
            
            server=df_shot1.iloc[0]["Player"]
            server_side=df_shot1.iloc[0]["Hit Zone"]
            if (is_replace_with_names):
                server=server.replace("Player", name1).replace("Opponent", name2)
            
            winner=df_shot1.iloc[0]["Winner_Point"]
            if (is_replace_with_names):
                #print(str(winner))
                if (str(winner)=="0"):
                    winner=name2
                if (str(winner)=="1"):
                    winner=name1

            df_shots_out_or_net = df_point[df_point["Result"].isin(["Out","Net"])]
            #if len(df_shots_out_or_net)>0:
            #    lst_shots_out = df_shots_out_or_net["Shot"].tolist()
            last_shot = df_point.iloc[-1]
    return (server, server_side, winner, df_shots_out_or_net, last_shot)
                                           
#print(get_point_infos(65, True, name1, name2))
def display_point_infos(index_point):
    #print(index_point)
    (server, server_side, winner, df_shots_out_or_net, last_shot)=(get_point_infos(index_point, True, name1, name2))
    lst_shots_out = []
    if (df_shots_out_or_net is not None):
        if (len(df_shots_out_or_net)>0):
            lst_shots_out = df_shots_out_or_net["Shot"].tolist()
            lst_shots_out = [str(shot) for shot in lst_shots_out]
    last_shot_str = ""
    time = ""
    #print(last_shot)
    if (last_shot is not None):
        last_shot_str = last_shot["Player"].replace("Player", name1).replace("Opponent", name2).upper()[0:3]
        last_shot_str += " " + last_shot["Result"]
        last_shot_str += "(" + str(last_shot["Shot"]) + ")"
        try:
            time = str(round(last_shot["Video Time"]))
        except:
            time=""
    print("Pt {0} ({6})- Srv: {1}*{5} - Won: {2} --- Last: {4} --- Shots OUT(or net): {3}".format(str(index_point),str(server).upper()[0:3], str(winner).upper()[0:3], ", ".join(lst_shots_out),last_shot_str, server_side, time))
#print(display_point_infos(1))

In [14]:
def update_df_result_bounce_hit(df, current_point, shot_index, new_bounce_x, new_bounce_y, new_hit_x, new_hit_y, shot_result):
    #change the value in the dataframe
    #set old result to resultOld
    df.loc[(df["Point"]==current_point)&(df["Shot"]==shot_index),"ResultOld"]=\
        df.loc[(df["Point"]==current_point)&(df["Shot"]==shot_index),"Result"]
    df.loc[(df["Point"]==current_point)&(df["Shot"]==shot_index),"Bounce (x)Old"]=\
        df.loc[(df["Point"]==current_point)&(df["Shot"]==shot_index),"Bounce (x)"]
    df.loc[(df["Point"]==current_point)&(df["Shot"]==shot_index),"Bounce (y)Old"]=\
        df.loc[(df["Point"]==current_point)&(df["Shot"]==shot_index),"Bounce (y)"]
    df.loc[(df["Point"]==current_point)&(df["Shot"]==shot_index),"Hit (x)Old"]=\
        df.loc[(df["Point"]==current_point)&(df["Shot"]==shot_index),"Hit (x)"]
    df.loc[(df["Point"]==current_point)&(df["Shot"]==shot_index),"Hit (y)Old"]=\
        df.loc[(df["Point"]==current_point)&(df["Shot"]==shot_index),"Hit (y)"]
    #set new result to Result
    df.loc[(df["Point"]==current_point)&(df["Shot"]==shot_index),"Result"]=shot_result
    df.loc[(df["Point"]==current_point)&(df["Shot"]==shot_index),"Bounce (x)"]=new_bounce_x
    df.loc[(df["Point"]==current_point)&(df["Shot"]==shot_index),"Bounce (y)"]=new_bounce_y
    df.loc[(df["Point"]==current_point)&(df["Shot"]==shot_index),"Hit (x)"]=new_hit_x
    df.loc[(df["Point"]==current_point)&(df["Shot"]==shot_index),"Hit (y)"]=new_hit_y
    #set new result to Result2
    df.loc[(df["Point"]==current_point)&(df["Shot"]==shot_index),"Result2"]=shot_result
    df.loc[(df["Point"]==current_point)&(df["Shot"]==shot_index),"Bounce (x)2"]=new_bounce_x
    df.loc[(df["Point"]==current_point)&(df["Shot"]==shot_index),"Bounce (y)2"]=new_bounce_y
    df.loc[(df["Point"]==current_point)&(df["Shot"]==shot_index),"Hit (x)2"]=new_hit_x
    df.loc[(df["Point"]==current_point)&(df["Shot"]==shot_index),"Hit (y)2"]=new_hit_y
    #set all next shots to 0
    if shot_result in ["Let","Out","Net"]:
        df.loc[(df["Point"]==current_point)&(df["Shot"]>shot_index),"Shot"]=0
        df.loc[(df["Point"]==current_point)&(df["Shot"]>shot_index),"Result2"]=shot_result

In [97]:
#GO TO NEXT POINT - last shot
current_point+=1
print(current_point)
go_to_nextpoint(player, current_point, -1)
#trace_point()

76
go_to_nextpoint 76


In [91]:
#GO TO PREV POINT - last shot
current_point-=1
#print(current_point)
go_to_nextpoint(player, current_point, -1)

go_to_nextpoint 70


In [17]:
#GO TO NEXT POINT - serve
current_point+=1
go_to_nextpoint(player, current_point, 1)
#trace_point()

go_to_nextpoint 1


In [18]:
#GO TO PREV POINT - serve
current_point-=1
#print(current_point)
go_to_nextpoint(player, current_point, 1)

go_to_nextpoint 0


In [19]:
hide_me
#https://www.cours-gratuit.com/tutoriel-python/tutoriel-python-matriser-la-bibliothque-ipywidgets
import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import display, Javascript
from IPython.display import clear_output

##DISPLAY box
def display_menu():
    display(
        widgets.VBox([widgets.HBox([btn_nextpoint,btn_prevpoint,btn_pause,btn_play, btn_plus_2sec, btn_minus_2sec, btn_end, btn_start])
                          , widgets.HBox([btn_nextpoint_last, btn_prevpoint_last, widget_shot_index,widget_lbl_point, widget_player])
                          , widgets.HBox([widget_bounce_x,widget_bounce_y, btn_update_shot, widget_shot_result])
                          , widgets.HBox([widget_hit_x,widget_hit_y, btn_cancel_point, btn_in_to_out])
                         ]))
    
#BUTTONS
btn_nextpoint = widgets.Button(description='Next Point', layout = widgets.Layout(width='160px'))
btn_prevpoint = widgets.Button(description='Prev Point', layout = widgets.Layout(width='160px'))
#display(btn_nextpoint)
btn_pause = widgets.Button(description='||', layout = widgets.Layout(width='130px'))
btn_play = widgets.Button(description='>', layout = widgets.Layout(width='60px'))
btn_end = widgets.Button(description='End', layout = widgets.Layout(width='60px'))
btn_plus_2sec = widgets.Button(description='+2s', layout = widgets.Layout(width='160px'))
btn_minus_2sec = widgets.Button(description='-2s', layout = widgets.Layout(width='160px'))
btn_start = widgets.Button(description='Start', layout = widgets.Layout(width='160px'))
btn_start = widgets.Button(description='Start', layout = widgets.Layout(width='130px'))
btn_cancel_point = widgets.Button(description='Cancel Pt', layout = widgets.Layout(width='130px'), style=dict(button_color="red"))
btn_in_to_out = widgets.Button(description='In->Out', layout = widgets.Layout(width='70px'), style=dict(button_color="gray"))
btn_nextpoint_last = widgets.Button(description='Next (last)', layout = widgets.Layout(width='150px'))
btn_prevpoint_last = widgets.Button(description='Prev (last)', layout = widgets.Layout(width='130px'))

def nextpoint_handler(btn_object):
    #print('You pressed the {} button!'.format(btn_object.description))
    clear_output()
    display_menu()
    btn_update_shot.description="Ok"
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()-2, IPython.notebook.get_selected_index()-1)'))
    display_point_infos(current_point+1)
    widget_lbl_point.value=current_point+1
def prevpoint_handler(btn_object):
    #print('You pressed the {} button!'.format(btn_object.description))
    clear_output()
    btn_update_shot.description="Ok"
    display_menu()
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()-1, IPython.notebook.get_selected_index())'))
    display_point_infos(current_point+1)
    widget_lbl_point.value=current_point+1
def pause_handler(btn_object):
    player.set_pause(1)
def play_handler(btn_object):
    player.set_pause(0)
def end_handler(btn_end):
    player.release()
def p2sec_handler(btn_object):
    time = player.get_time()
    player.set_time(time+2000)
def m2sec_handler(btn_object):
    time = player.get_time()
    player.set_time(time-2000)
def start_handler(btn_start):
    # call the video method
    player = video2(filename)
    #GO to point 1
    current_point=0
def cancel_handler(btn_object):
    #point = widget_lbl_point.value
    print("hello")
    print("cancel " + str(current_point))
    df.loc[(df["Point"]==current_point),"Shot"]=0
    df.loc[(df["Point"]==current_point),"comment"]="cancelled"
    print(df.loc[(df["Point"]==current_point),"Shot"])
    df.to_csv(filename_csv)
def in_to_out_handler(btn_object):
    df_shot=df[(df["Point"]==current_point)&(df["Shot"]==widget_shot_index.value)].head()
    x,y=change_shot_in_to_out(widget_bounce_x.value, widget_bounce_y.value
                               , df_shot.iloc[0]["Shot"]
                               , df_shot.iloc[0]["Hit Zone"]
                               , df_shot.iloc[0]["Hit Side"])
    widget_bounce_x.value=x
    widget_bounce_y.value=y
    
#player = btn_start.on_click(start_handler)
btn_nextpoint.on_click(nextpoint_handler)
btn_prevpoint.on_click(prevpoint_handler)
btn_pause.on_click(pause_handler)
btn_play.on_click(play_handler)
btn_end.on_click(end_handler)
btn_plus_2sec.on_click(p2sec_handler)
btn_minus_2sec.on_click(m2sec_handler)
btn_start.on_click(start_handler)
btn_cancel_point.on_click(cancel_handler)
btn_in_to_out.on_click(in_to_out_handler)

#LABEL Pt
widget_lbl_point = widgets.Label(value='Pt ', layout = widgets.Layout(width='130px'))
def set_label_point(current_point):
    widget_lbl_point.value="Pt " + str(current_point)
    
def nextpoint_last_handler(btn_object):
    #print('You pressed the {} button!'.format(btn_object.description))
    clear_output()
    display_menu()
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()-4, IPython.notebook.get_selected_index()-3)'))
    btn_update_shot.description="Ok"
    player.set_pause(1)
    display_point_infos(current_point+1)
    trace_point(current_point+1)
    set_label_point(current_point+1)
    #last shot infos
    (server, server_side, winner, df_shots_out_or_net, last_shot) = get_point_infos(current_point+1, True, name1, name2)
    if (last_shot is not None):
        print(last_shot["Shot"])
        widget_shot_index.value = last_shot["Shot"]
        init_inputvalues_bounces_and_result(last_shot["Shot"], current_point+1)
    else:
        widget_shot_index.value = 0
    time.sleep(1)
    player.set_pause(0)

def prevpoint_last_handler(btn_object):
    #print('You pressed the {} button!'.format(btn_object.description))
    clear_output()
    display_menu()
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()-3, IPython.notebook.get_selected_index()-2)'))
    btn_update_shot.description="Ok"
    display_point_infos(current_point+1)
    trace_point(current_point-1)
    set_label_point(current_point+1)
    #last shot infos
    (server, server_side, winner, df_shots_out_or_net, last_shot) = get_point_infos(current_point+1, True, name1, name2)
    if (last_shot is not None):
        widget_shot_index.value = last_shot["Shot"]
        init_inputvalues_bounces_and_result(last_shot["Shot"], current_point+1)
    else:
        widget_shot_index.value = 0
        init_inputvalues_bounces_and_result(0, current_point+1)
    
#BTN next_last, prev_last
btn_nextpoint_last.on_click(nextpoint_last_handler)
btn_prevpoint_last.on_click(prevpoint_last_handler)

def init_inputvalues_bounces_and_result(shot_index, current_point):
    #print("init_inputvalues_bounces_and_result" + str(shot_index))
    df_shot=df[(df["Point"]==current_point)&(df["Shot"]==shot_index)].head()
    if len(df_shot)>0:
        #set INPUTs bounce x/y to the value of the current shot
        widget_bounce_x.value=round(df_shot.iloc[0]["Bounce (x)"], 2)
        widget_bounce_y.value=round(df_shot.iloc[0]["Bounce (y)"], 2)
        widget_hit_x.value=round(df_shot.iloc[0]["Hit (x)"], 2)
        widget_hit_y.value=round(df_shot.iloc[0]["Hit (y)"], 2)
        #set dropdown to the current shot value
        widget_shot_result.value=df_shot.iloc[0]["Result"]
        #set dropdown player
        if df_shot.iloc[0]["Player"]=="Player":
            widget_player.value=name1
        elif df_shot.iloc[0]["Player"]=="Opponent":
            widget_player.value=name2
        else:
            widget_player.value="None"
    else:
        widget_bounce_x.value=0
        widget_bounce_y.value=0
        widget_hit_x.value=0
        widget_hit_y.value=0
        widget_shot_result.value="None"
        widget_player.value="None"
        
#SHOT INPUT
def on_change_shot(change):
    if change['type'] == 'change' and change['name'] == 'value':
        shot_index=change['new']
        init_inputvalues_bounces_and_result(shot_index, current_point)
        
widget_shot_index=widgets.IntText(value=0,description='Shot', disabled=False, layout = widgets.Layout(width='130px'))
widget_shot_index.observe(on_change_shot) 
widget_shot_index.add_class("left-spacing-class")

#PLAYER DROPDWON
widget_player = widgets.Dropdown(
    options=["None",name1, name2],
    value="None",
    description='By:',
    disabled=False,
    layout = widgets.Layout(width='150px')
)
#BOUNCE        
widget_bounce_x=widgets.FloatText(description='X', disabled=False, layout = widgets.Layout(width='160px'), description_tooltip="-4.11 4.11")
widget_bounce_y=widgets.FloatText(description='Y', disabled=False, layout = widgets.Layout(width='160px'), description_tooltip="0- 5.48-Serv-11.88\n11.88-Serve-18.48 - 23.73")
widget_bounce_y.add_class("left-spacing-class")
widget_bounce_x.add_class("left-spacing-class")
widget_hit_x=widgets.FloatText(description='hit X', disabled=False, layout = widgets.Layout(width='160px'), description_tooltip="-4.11 4.11")
widget_hit_y=widgets.FloatText(description='hit Y', disabled=False, layout = widgets.Layout(width='160px'), description_tooltip="0- 5.48-Serv-11.88\n11.88-Serve-18.48 - 23.73")
widget_hit_y.add_class("left-spacing-class")
widget_hit_x.add_class("left-spacing-class")
#RESULT RadioButtons

#BUTTON UPDATE SHOT
btn_update_shot = widgets.Button(description='Ok', layout = widgets.Layout(width='100px'), style=dict(button_color="green"))

def update_shot_handler(btn_object):
    print("A")
    shot_result = widget_shot_result.value#change['new']
    if shot_result == "None":
        return
    elif shot_result == "Swap Srv":
        swap_server_all_shots(df, current_point)
    elif shot_result == "Shot=0":
        shot_index=widget_shot_index.value
        df.loc[(df["Point"]==current_point)&(df["Shot"]==shot_index),"Shot"]=0
        df.loc[(df["Point"]==current_point)&(df["Shot"]==shot_index),"Result2"]=shot_result
    else:
        shot_index=widget_shot_index.value
        print("Shot {0}-new Result:{1}".format(shot_index, shot_result))
        print(df[(df["Point"]==current_point)&(df["Shot"]==shot_index)].head())
        update_df_result_bounce_hit(df, current_point, shot_index, widget_bounce_x.value, widget_bounce_y.value
                                 , widget_hit_x.value, widget_hit_y.value, shot_result)
    set_point_winner(df, current_point)
    df.to_csv(filename_csv)
    btn_update_shot.description="-"

btn_update_shot.on_click(update_shot_handler)    
#btn_update_shot.add_class("left-spacing-class")
display(HTML(
     "<style>.left-spacing-class {margin-left: -5px;}</style>"
))
#RADIO
widget_shot_result=widgets.ToggleButtons(
    options=["None",'In', 'Out', 'Net', 'Let', "Shot=0","Swap Srv"],
    value='None',
    description='',
    disabled=False,
    button_color=["red","red","red","red","red","red","red"],
    style={"button_width": "65px", 'button_color':"red"}#, layout={'width': '280px'}
)
#widget_shot_result.observe(on_change_result) 

display_menu()

<IPython.core.display.Javascript object>

Pt 76 (2018)- Srv: ISN*ad - Won: POS --- Last: ISN Net(5) --- Shots OUT(or net): 1, 4, 5
trace_point 75


5


In [20]:
def check_server_sequences():
    #FOR EACH UNIQUE POINT, set point_winner for all shots of this point
    for index_point in df.sort_values(by=["Point","Shot"])["Point"].unique():
        (server, server_side, winner, df_shots_out_or_net, last_shot)=(get_point_infos(index_point, True, name1, name2))
        #display_point_infos(point_index)
        print("Point {1} - {0}".format(server,index_point))
check_server_sequences()

Point 1 - Pospisil
Point 2 - Pospisil
Point 3 - 
Point 4 - 
Point 5 - Pospisil
Point 7 - Isner
Point 9 - Isner
Point 10 - Isner
Point 12 - Isner
Point 13 - Isner
Point 14 - 
Point 15 - Pospisil
Point 17 - Pospisil
Point 18 - Pospisil
Point 19 - Pospisil
Point 20 - 
Point 22 - Isner
Point 25 - Isner
Point 26 - Isner
Point 27 - 
Point 28 - Isner
Point 29 - 
Point 30 - Isner
Point 31 - 
Point 32 - Isner
Point 33 - Pospisil
Point 34 - Pospisil
Point 36 - 
Point 37 - Pospisil
Point 38 - 
Point 41 - Pospisil
Point 42 - Pospisil
Point 43 - Pospisil
Point 45 - Pospisil
Point 47 - Pospisil
Point 48 - Pospisil
Point 49 - Isner
Point 51 - Isner
Point 53 - Isner
Point 54 - 
Point 55 - Isner
Point 57 - Pospisil
Point 58 - Pospisil
Point 60 - 
Point 63 - Pospisil
Point 65 - Isner
Point 67 - Isner
Point 68 - 
Point 69 - Isner
Point 71 - Isner
Point 72 - 
Point 73 - Isner
Point 75 - Pospisil
Point 76 - Isner
Point 78 - 
Point 79 - Pospisil
Point 81 - Pospisil
Point 82 - Pospisil
Point 83 - Isner
Point

In [21]:
current_point=75